# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
# List all collections in the database
print(db.list_collection_names())


['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
hygiene_20_count = establishments.count_documents(query)
print("Number of establishments with a hygiene score of 20:", hygiene_20_count)


# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)


Number of establishments with a hygiene score of 20: 41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': {'$toInt': '$RatingValue'},
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('665dee4be9ee1db431f09df0'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',


In [7]:
# Convert the result to a Pandas DataFrame
cursor = establishments.find(query)
df = pd.DataFrame(list(cursor))

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))


# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))


Number of rows in the DataFrame: 41
First 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  665dee4be9ee1db431f09df0   110681                  0   
1  665dee4be9ee1db431f0a171   612039                  0   
2  665dee4be9ee1db431f0a479   730933                  0   
3  665dee4be9ee1db431f0a669   172735                  0   
4  665dee4be9ee1db431f0a675   172953                  0   
5  665dee4ce9ee1db431f0b017   512854                  0   
6  665dee4ce9ee1db431f0b238  1537089                  0   
7  665dee4ce9ee1db431f0c764   155648                  0   
8  665dee4ce9ee1db431f0cba8  1012883                  0   
9  665dee4ce9ee1db431f0d3b7   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza 

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"$and": [{"LocalAuthorityName": {"$regex": "London", "$options": "i"}}, {"RatingValue": {"$gte": 4}}]}

# Use count_documents to display the number of documents in the result
rating_4_london_count = establishments.count_documents(query)
print("Number of establishments in London with RatingValue greater than or equal to 4:", rating_4_london_count)

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

Number of establishments in London with RatingValue greater than or equal to 4: 0
None


In [9]:
# Convert the result to a Pandas DataFrame
cursor = establishments.find(query)
df = pd.DataFrame(list(cursor))

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))


Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [10]:
# Set the latitude and longitude for searching within 0.01 degree
degree_search = 0.01
latitude = 51.490142  # Latitude of Penang Flavours
longitude = 0.083840  # Longitude of Penang Flavours

# Define the query to find establishments with RatingValue = 5 within 0.01 degree of latitude and longitude
query = {
    "$and": [
        {"geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search}},
        {"geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}},
        {"RatingValue": 5}
    ]
}

# Define the sort criteria to sort by hygiene score in ascending order
sort = [("scores.Hygiene", 1)]

# Define the limit to get only the top 5 establishments
limit = 5

# Execute the query and sort the results
cursor = establishments.find(query).sort(sort).limit(limit)

# Convert the cursor to a Pandas DataFrame
df = pd.DataFrame(list(cursor))

# Print the results
print(df)

Empty DataFrame
Columns: []
Index: []


In [11]:
# Execute the query and sort the results
cursor = establishments.find(query).sort(sort).limit(limit)

# Convert the cursor to a Pandas DataFrame
df = pd.DataFrame(list(cursor))

# Print the DataFrame
print(df)


Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [12]:
# Define the aggregation pipeline
pipeline = [
    # Match establishments with a hygiene score of 0
    {"$match": {"scores.Hygiene": 0}},
    # Group the matches by Local Authority
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    # Sort the matches from highest to lowest
    {"$sort": {"count": -1}}
]

# Execute the aggregation pipeline
result = establishments.aggregate(pipeline)

# Convert the result to a list and print the number of documents
result_list = list(result)
print("Number of documents in the result:", len(result_list))

# Print the first 10 results
print("First 10 results:")
for idx, item in enumerate(result_list[:10]):
    print(f"{idx+1}. {item['_id']}: {item['count']}")


Number of documents in the result: 55
First 10 results:
1. Thanet: 1130
2. Greenwich: 882
3. Maidstone: 713
4. Newham: 711
5. Swale: 686
6. Chelmsford: 680
7. Medway: 672
8. Bexley: 607
9. Southend-On-Sea: 586
10. Tendring: 542


In [13]:
# Execute the aggregation pipeline
result = establishments.aggregate(pipeline)

# Convert the result to a list
result_list = list(result)

# Convert the list to a Pandas DataFrame
df = pd.DataFrame(result_list)

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))


Number of rows in the DataFrame: 55
First 10 rows of the DataFrame:
               _id  count
0           Thanet   1130
1        Greenwich    882
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542
